# 05: Geometric Hashing

There is a general problem in the study of networks or graphs, where we have to discover all pairs of objects in a set which are close to each other, for some cutoff distance $r_{cut}$ defined by embedding the objects in some kind of space.

For this week's example we will look at some atoms in a box in $\mathbb{R}^3$, but this is a very general question, we could be talking about cities on a map, mobile phone base stations, etc.

The basic algorithm is to loop once over the atoms and place each one in a "cell" defined by its local neighbourhood.  To find neighbours we only need to compare atoms in the same cell, or in one of the $3^3-1=26$ neighbouring cells because any pair not in adjacent cells (including corners) will automatically be too far away.  This gets us a speedup from $\mathcal{O}(N^2)$ to $\mathcal{O}(N)$, which is very valuable when calculating short-range interactions between atoms in realistic computer modelling of materials.



In [1]:
#include <stdio.h>
#include <stdlib.h>

//Define a (listable) structured type ATOM_T to hold info about a (classical) atom
//I've simplified the datatype to only contain info that is needed for today's problem
typedef struct atom_t {
    atom_t   *listNext;      //can add to a list of similar objects
    double    position[3];   
} ATOM_T; 



In [2]:
ATOM_T *newAtom( double x, double y, double z ){
    
    /* create an atom in a given position */
    
    ATOM_T *a;
    
    a = (ATOM_T *)malloc( sizeof(ATOM_T) );
    a->listNext     = NULL;
    a->position[0]  = x;
    a->position[1]  = y;
    a->position[2]  = z;
    
    return( a );
}


In [3]:
ATOM_T **boxOfAtoms(int N_atoms, double box_L, int seed ){
    
    /*
     create some atoms and place them randomly in a box centred at the origin.
    */
    ATOM_T **atoms;
    int          i;
    double  half_L;
    
    half_L = L * 0.5;
    
    //init the random number generator, so that code is repeatable
    srand( seed );
    
    atoms = (ATOM_T **)malloc(N_atoms*sizeof(ATOM_T *));
    for( i = 0; i < N_atoms; i++ ){
        atoms[i] = newAtom( (rand()*(box_L/RAND_MAX)) - half_L,
                            (rand()*(box_L/RAND_MAX)) - half_L,
                            (rand()*(box_L/RAND_MAX)) - half_L );
        
    }
    return( atoms );
}

In [4]:
double check_contact( ATOM_T* a, ATOM_T* b, double box_L, double r_cut ){
    /* 
    check if two atoms are in "contact" (closer than distance r_cut)
    
    subject to periodic boundary conditions
    */
    double dx[3], r2, half_L;
    int    d;
    
    half_L = box_L * 0.5;
    for( d = 0; d < 3; d++ ){
      //displacement vector a to b.
      dx[d] = b->position[d] - a->position[d];
        
      //periodic imaging, now see nearest image.
      if     ( dx[d] >  half_L ) dx[d] -= box_L;
      else if( dx[d] < -half_L ) dx[d] += box_L;
    }
    
    r2 = dx[0]*dx[0] + dx[1]*dx[1] + dx[2]*dx[2]; 
    
    //return r if we are in contact, otherwise -1 (impossible r).
    if( r2 > r_cut*r_cut ) return( -1.0 );
    return( sqrt(r2) );
    
}

In [2]:
void assignCellIndex( double *posn, int *ix, int *iy, int *iz, double box_L, double r_cell ){
    /* assign cell indices for points on [-0.5*box_L.. 0.5*box_L) 
    
       This is done by integer rounding-down, assumes that the 
       box is centred at the origin and that all particles are
       in the box.   [-L/2 <= x,y,z < L/2)
    */
    double half_L;
    
    half_L = box_L * 0.5;
    
    //write to the output variables provided by the pointers ix, iy, iz
   *ix = (int)((posn[0]+half_L)/r_cell);
   *iy = (int)((posn[1]+half_L)/r_cell);
   *iz = (int)((posn[2]+half_L)/r_cell);
}

OK great I have given you some code to 

1) Create a box of atoms

2) See if two atoms are in contact

3) See which neighbour cell (by 3D array indices) a given atom should be in

Before I do the whole exercise I am going to stop coding and ask you to take over.


## Assignment, week 05: Cells and neighbours

Write programs (create new cells below) which do the following, each one should make a printout to show that it is working:

1) Place $N$ atoms in a cubic periodic box of side-length $L$ and count the number of pairs of atoms less than $r_{cut}$ distance unit apart. Your code should use geometric hashing such that it will have linear scaling in the limit of large number of atoms.  Make it easy for me to change $N,L,r_{cut}$ when I test.

In the above implementation remember: 

1) You can track which atoms are in a cell by making linked lists.

2) You will need an integer number of cells.  Optimal cell size is $r_{cut}$, how do you adapt this?

3) Your code should still work if the box is less than $3r_{cut}$ in size.